## 복합체인 : 나라설명 -> 그 나라에서 제일 유명한 음식(food_chain)
##               음식 -> 레시피(recipe_chain)

* lim 모델을 생성

In [2]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
result = llm.invoke("What is the capital of Korea?")
result

ModuleNotFoundError: No module named 'langchain_ollama'

In [5]:
from langchain_core.prompts import PromptTemplate
llm = ChatOllama(model="llama3.2:1b")
prompt_template = PromptTemplate(
    template="What is the capital of {country}?", # {}안의 값을 새로운 값으로 대입 가능
    input_variables = ["country"]
)

prompt = prompt_template.invoke({"country":"Korea"})
print(prompt)
llm.invoke(prompt)

text='What is the capital of Korea?'


AIMessage(content="The capital of South Korea is Seoul. However, it's worth noting that North Korea also claims Seoul as its capital, and the city has a complex history with both countries claiming sovereignty over it.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-10T02:04:05.7298741Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1607088700, 'load_duration': 131176200, 'prompt_eval_count': 32, 'prompt_eval_duration': 36355200, 'eval_count': 40, 'eval_duration': 1404999000, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b0600-5698-7eb0-a868-91698ed717ed-0', usage_metadata={'input_tokens': 32, 'output_tokens': 40, 'total_tokens': 72})

* 음식 추천 invoke 구현

In [32]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="대한민국에서 제일 유명한 음식은?"), # 모범질문
    AIMessage(content="비빔밥"),    # 모범답안
]
llm.invoke(message_list)

AIMessage(content='입니다. 비빔밥은 한국의 대표적인 음식 중 하나이며, 다양한 조각으로 구성되어 있습니다. 이 음식은 한국의 전통อาหาร 중 하나로, 다양한 조각으로 구성된 비빔밥은 한국의 식사 문화에서 매우 phổ biến합니다.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-10T02:44:51.1282536Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2607260800, 'load_duration': 106510100, 'prompt_eval_count': 48, 'prompt_eval_duration': 283336800, 'eval_count': 63, 'eval_duration': 2157616000, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b0625-a307-7692-9a03-0a594ae6a605-0', usage_metadata={'input_tokens': 48, 'output_tokens': 63, 'total_tokens': 111})

* 다양한 답변 형식을 컨트롤할 수 있도록 음식 추천 구현

In [16]:
from pydantic import BaseModel, Field
class User(BaseModel):
    # gt=0:id>0 / ge=0:id>=0 / lt=0:id<0 / le=0:id<=0
    id:int   = Field(gt=0,         description="id")
    name:str = Field(min_length=2, description="name")
    is_active:bool = Field(default=True, description="id활성화 여부")
user = User(id="1", name="비빔밥")
print(user)
country_detail_prompt = PromptTemplate(
    template = """Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in JSON format and return the JSON dictionary only""",
    input_variables=["country"]
)
class CountryDetail(BaseModel): #description: 더 정확한 출력 유도
    capital:str  = Field(description="대한민국에서 제일 유명한 음식은?")
    population:int = Field(description="비빔밥이 제일 맛있나요?")
# 출력 형식 파서 + LLM
structedllm = llm.with_structured_output(CountryDetail)
# llm.invoke(country_detail_prompt.invoke({"country":"Korea"}))
info = structedllm.invoke(country_detail_prompt.invoke({"country":"Korea"}))
info

id=1 name='비빔밥' is_active=True


CountryDetail(capital='Seoul', population=51000000)

* 레시피 추천 invoke 구현

In [24]:
# 나라 설명 -> 나라이름
country_prompt = PromptTemplate(
    template="""다음의 {information} 설명을 보고 나라이름을 맞춰봐:
    {information}
    나라 이름만 한국어로 reutrn 해 줘""",
    input_variables=["information"]
)
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":
                            "이 나라는 비빔밥으로 유명해"})))

'이 나라는 비빔밥으로 유명한 나 Rai름은:\n\n   오스트리아'

* 다양한 답변 형식을 컨트롤 할 수 있도록 레시피 생성형 AI 구현

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model="llama3.2:1b",
                temperature=0) # 일관된 답변(보수적인 답변)
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Retrun the name of the city only.",
    input_variables = ["country"]
)
output_parser = StrOutputParser() # AIMessage()를 Str변환
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

* 위의두 렝체인을 연결

In [26]:
# 프롬프트 템플릿 -> llm -> 출력파서를 연결하는 체인 생성
capital_chain = prompt_template | llm | output_parser
# 생성된 체인 invoke
capital_chain.invoke({"country":"Korea"})

'Seoul'

* 최종 렝체인을 연결

In [31]:
# 복합체인 : information -> country_chain -> (나라명을 country) -> capital_chain
from langchain_core.runnables import RunnablePassthrough
final_chain = {"information":RunnablePassthrough()} | \
                {"country":country_chain} | capital_chain
final_chain.invoke("This country is very famous for its wine")

'Seoul'

* 과제물에 적절할 파일 제출 여부

* 시간엄수

* 적절할 모델 선택